In [0]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVR
from sklearn import metrics
from sklearn.metrics import accuracy_score
import pickle


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
url = 'https://raw.githubusercontent.com/tna-hub/Cloud-TP2/master/dataset.csv'
data = pd.read_csv(url)
data.head()


,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,6.0,18.0,5.0,NaN,800.0,88.0,0.1,1021.1,-18.6,0.0,NW,4.4,Gucheng
1,2,2013,3,1,1,6.0,15.0,5.0,NaN,800.0,88.0,-0.3,1021.5,-19.0,0.0,NW,4.0,Gucheng
2,3,2013,3,1,2,5.0,18.0,NaN,NaN,700.0,52.0,-0.7,1021.5,-19.8,0.0,WNW,4.6,Gucheng
3,4,2013,3,1,3,6.0,20.0,6.0,NaN,NaN,NaN,-1.0,1022.7,-21.2,0.0,W,2.8,Gucheng
4,5,2013,3,1,4,5.0,17.0,5.0,NaN,600.0,73.0,-1.3,1023.0,-21.4,0.0,WNW,3.6,Gucheng


In [0]:
data.isnull().sum()  # showing the number of rows with null values


No             0
year           0
month          0
day            0
hour           0
PM2.5       8043
PM10        5965
SO2         8352
NO2        11362
CO         19404
O3         12199
TEMP         379
PRES         374
DEWP         384
RAIN         371
wd          1743
WSPM         305
station        0
dtype: int64

In [0]:
data = data.dropna()  # drop all rows with null values
data.isnull().sum()  # verify it worked 

No         0
year       0
month      0
day        0
hour       0
PM2.5      0
PM10       0
SO2        0
NO2        0
CO         0
O3         0
TEMP       0
PRES       0
DEWP       0
RAIN       0
wd         0
WSPM       0
station    0
dtype: int64

In [0]:
data.describe()  # get more info about data

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
count,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000,349400.000000
mean,17969.317490,2014.716657,6.472350,15.711829,11.579379,78.981647,103.885010,15.492473,50.125805,1217.505690,57.513042,13.494827,1010.743455,2.399353,0.065279,1.736347
std,9998.752658,1.159938,3.449074,8.803836,6.934095,79.712207,90.840935,21.055879,34.969247,1150.170175,56.676463,11.430139,10.440024,13.805684,0.827349,1.244607
min,1.000000,2013.000000,1.000000,1.000000,0.000000,2.000000,2.000000,0.285600,2.000000,100.000000,0.214200,-19.900000,982.400000,-36.000000,0.000000,0.000000
25%,9629.000000,2014.000000,3.000000,8.000000,6.000000,20.000000,36.000000,2.000000,23.000000,500.000000,11.000000,3.100000,1002.300000,-9.000000,0.000000,0.900000
50%,18120.000000,2015.000000,6.000000,16.000000,12.000000,54.000000,81.000000,7.000000,43.000000,900.000000,45.000000,14.400000,1010.300000,2.900000,0.000000,1.400000
75%,26522.000000,2016.000000,9.000000,23.000000,18.000000,110.000000,145.000000,19.000000,70.000000,1500.000000,82.000000,23.200000,1019.000000,15.100000,0.000000,2.200000
max,35064.000000,2017.000000,12.000000,31.000000,23.000000,844.000000,999.000000,500.000000,290.000000,10000.000000,1071.000000,41.600000,1042.800000,29.100000,72.500000,12.900000


In [0]:
labelencoder = LabelEncoder()
data['wd'] = labelencoder.fit_transform(data['wd'])
data['station'] = labelencoder.fit_transform(data['station'])


In [0]:
data

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
20,21,2013,3,1,20,13.0,25.0,12.0,5.0,1100.0,61.0,1.6,1027.1,-18.4,0.0,2,1.9,5
21,22,2013,3,1,21,15.0,23.0,14.0,13.0,1200.0,52.0,1.0,1028.1,-17.4,0.0,10,0.7,5
22,23,2013,3,1,22,16.0,28.0,16.0,19.0,1200.0,45.0,1.3,1028.4,-17.6,0.0,0,1.0,5
23,24,2013,3,1,23,16.0,28.0,14.0,20.0,1100.0,44.0,0.2,1028.6,-17.6,0.0,2,1.1,5
24,25,2013,3,2,0,14.0,18.0,24.0,43.0,1399.0,25.0,-0.3,1028.9,-18.0,0.0,1,1.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385699,35060,2017,2,28,19,11.0,27.0,4.0,20.0,300.0,81.0,12.6,1011.9,-14.3,0.0,3,2.0,10
385700,35061,2017,2,28,20,15.0,43.0,6.0,55.0,500.0,45.0,9.4,1012.3,-11.9,0.0,15,1.0,10
385701,35062,2017,2,28,21,13.0,35.0,7.0,48.0,500.0,48.0,8.7,1012.8,-13.7,0.0,3,1.1,10
385702,35063,2017,2,28,22,12.0,31.0,5.0,47.0,500.0,50.0,7.8,1012.9,-12.6,0.0,5,1.0,10


In [0]:
data.corr()  # check for highest correlation values

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
No,1.000000,0.968604,0.056135,0.017161,-0.003799,-0.029323,-0.049093,-0.226194,-0.043095,0.028447,-0.049297,-0.134103,0.186158,-0.111080,0.005317,-0.021251,0.055228,-0.005576
year,0.968604,1.000000,-0.192972,-0.008328,-0.003975,-0.032162,-0.043294,-0.166929,-0.052502,0.016569,-0.023017,-0.168096,0.188252,-0.179834,0.000920,-0.012707,0.090281,-0.006688
month,0.056135,-0.192972,1.000000,0.018011,-0.001608,0.012734,-0.022081,-0.224337,0.039098,0.046540,-0.101499,0.147863,-0.025851,0.285297,0.017748,-0.033399,-0.144272,0.004666
day,0.017161,-0.008328,0.018011,1.000000,-0.002192,0.003604,0.023292,-0.007298,0.012910,-0.018428,-0.000920,0.011928,0.023196,0.023730,-0.002491,0.002133,-0.013400,0.001787
hour,-0.003799,-0.003975,-0.001608,-0.002192,1.000000,0.013606,0.051150,0.033588,0.000951,-0.029654,0.280606,0.139736,-0.038567,-0.012335,0.013119,0.164512,0.137427,0.005675
PM2.5,-0.029323,-0.032162,0.012734,0.003604,0.013606,1.000000,0.884017,0.479773,0.667725,0.789387,-0.146996,-0.124145,0.012171,0.121528,-0.014920,-0.079139,-0.273450,0.026361
PM10,-0.049093,-0.043294,-0.022081,0.023292,0.051150,0.884017,1.000000,0.461051,0.650845,0.699844,-0.111283,-0.090685,-0.023393,0.075048,-0.026542,-0.039990,-0.181472,0.022254
SO2,-0.226194,-0.166929,-0.224337,-0.007298,0.033588,0.479773,0.461051,1.000000,0.501781,0.532080,-0.164773,-0.318438,0.221443,-0.264875,-0.040567,-0.018950,-0.106464,0.004933
NO2,-0.043095,-0.052502,0.039098,0.012910,0.000951,0.667725,0.650845,0.501781,1.000000,0.701348,-0.472250,-0.274383,0.175303,-0.029074,-0.043883,-0.118394,-0.396123,0.081273
CO,0.028447,0.016569,0.046540,-0.018428,-0.029654,0.789387,0.699844,0.532080,0.701348,1.000000,-0.313439,-0.323086,0.185726,-0.055313,-0.013169,-0.126399,-0.294312,0.035658


In [0]:
#  split the data into training and test dataset - 80 % - train, 20 % - test

In [0]:
columns = ["year", "month", "hour", "PM2.5", "PM10", "SO2", "NO2", "CO", "O3", "PRES", "DEWP", "wd"]

norm_mean_dict = {}
norm_stdev_dict = {}

for column in columns:
    norm_stdev_dict.update( {column : int(data[column].std()) } )
    norm_mean_dict.update( {column : int(data[column].mean()) } )
    data[column] = (data[column] - data[column].mean()) / data[column].std() + 3


In [0]:
norm_mean_dict

{'CO': 1217,
 'DEWP': 2,
 'NO2': 50,
 'O3': 57,
 'PM10': 103,
 'PM2.5': 78,
 'PRES': 1010,
 'SO2': 15,
 'hour': 11,
 'month': 6,
 'wd': 6,
 'year': 2014}

In [0]:
norm_stdev_dict

{'CO': 1150,
 'DEWP': 13,
 'NO2': 34,
 'O3': 56,
 'PM10': 90,
 'PM2.5': 79,
 'PRES': 10,
 'SO2': 21,
 'hour': 6,
 'month': 3,
 'wd': 4,
 'year': 1}

In [0]:
data

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
20,21,1.520044,1.993252,1,4.214379,2.172252,2.131614,2.834133,1.709557,2.897836,3.061524,1.6,4.566715,1.493421,0.0,1.956500,1.9,5
21,22,1.520044,1.993252,1,4.358594,2.197342,2.109597,2.929118,1.938330,2.984780,2.902728,1.0,4.662500,1.565855,0.0,3.724852,0.7,5
22,23,1.520044,1.993252,1,4.502809,2.209887,2.164639,3.024104,2.109909,2.984780,2.779220,1.3,4.691236,1.551368,0.0,1.514412,1.0,5
23,24,1.520044,1.993252,1,4.647024,2.209887,2.164639,2.929118,2.138506,2.897836,2.761576,0.2,4.710393,1.551368,0.0,1.956500,1.1,5
24,25,1.520044,1.993252,2,1.330081,2.184797,2.054556,3.404045,2.796227,3.157798,2.426340,-0.3,4.739129,1.522395,0.0,1.735456,1.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385699,35060,4.968504,1.703319,28,4.070164,2.147161,2.153630,2.454192,2.138506,2.202287,3.414404,12.6,3.110780,1.790400,0.0,2.177544,2.0,10
385700,35061,4.968504,1.703319,28,4.214379,2.197342,2.329762,2.549177,3.139385,2.376174,2.779220,9.4,3.149094,1.964242,0.0,4.830072,1.0,10
385701,35062,4.968504,1.703319,28,4.358594,2.172252,2.241696,2.596670,2.939209,2.376174,2.832152,8.7,3.196987,1.833861,0.0,2.177544,1.1,10
385702,35063,4.968504,1.703319,28,4.502809,2.159707,2.197663,2.501684,2.910613,2.376174,2.867440,7.8,3.206565,1.913538,0.0,2.619632,1.0,10


In [0]:
#x = data[["year", "month", "day", "hour", "PM2.5", "PM10", "SO2", "NO2", "CO", "O3", "PRES", "DEWP", "RAIN", "wd", "WSPM", "station"]]
x = data[["year", "month", "hour", "PM2.5", "PM10", "SO2", "NO2", "CO", "O3", "PRES", "DEWP", "wd"]]
y = data["TEMP"]
train_data = pd.DataFrame()
test_data = pd.DataFrame()
train_data, test_data, train_label, test_label = train_test_split(x, y, test_size=0.2)



In [0]:
# Encode sequentially the string values

In [0]:
#labelencoder = LabelEncoder()
#train_data['wd'] = labelencoder.fit_transform(train_data['wd'])
#train_data['station'] = labelencoder.fit_transform(train_data['station'])
#test_data['wd'] = labelencoder.fit_transform(test_data['wd'])
#test_data['station'] = labelencoder.fit_transform(test_data['station'])


In [0]:
train_data  # verify it worked

,year,month,hour,PM2.5,PM10,SO2,NO2,CO,O3,PRES,DEWP,wd
227937,3.244274,1.993252,2.628015,2.749328,2.538919,2.881626,3.167982,3.245611,2.055815,2.277448,2.181543,2.840676
65156,4.106389,3.442916,4.214379,2.711692,2.472870,2.359206,2.167102,2.550062,5.284669,2.354077,4.622567,3.503808
294427,2.382159,4.022782,4.070164,2.661512,2.428837,2.359206,2.624647,2.897836,2.126391,3.062887,3.543301,2.840676
253376,2.382159,1.413386,2.483800,2.122071,2.098589,3.166582,2.653244,2.376174,2.655712,4.145260,1.428231,3.061720
22372,3.244274,3.732849,1.906940,2.460790,2.461861,2.406699,3.396754,2.463118,2.091103,2.995838,3.825794,3.061720
...,...,...,...,...,...,...,...,...,...,...,...,...
215699,1.520044,4.022782,2.916445,2.423154,2.296738,2.834133,2.052716,2.115343,2.585136,3.082044,3.514328,3.282764
126056,3.244274,3.152983,2.483800,2.209887,2.065564,2.359206,1.823944,2.115343,2.973304,1.798522,4.267641,1.514412
343621,4.106389,2.573117,3.204875,2.310248,2.175647,2.359206,1.938330,2.202287,3.432048,2.928788,2.746528,3.061720
40273,1.520044,4.022782,1.474296,2.523515,2.362787,3.071596,2.882016,2.810893,2.461628,2.928788,3.673682,1.735456


In [0]:
test_data  

,year,month,hour,PM2.5,PM10,SO2,NO2,CO,O3,PRES,DEWP,wd
252784,1.520044,4.602648,3.637520,2.372974,2.968241,2.454192,2.167102,2.202287,3.043880,2.775532,1.667262,3.061720
72226,1.520044,2.573117,2.772230,2.473335,2.098589,2.739148,1.938330,2.202287,3.096812,1.980514,3.898228,4.830072
112627,2.382159,1.413386,4.070164,3.903480,3.276472,3.071596,3.596930,3.767273,2.002883,3.082044,2.377115,1.735456
274217,4.106389,2.863050,3.781734,2.096981,2.175647,2.359206,1.852540,2.202287,4.190740,2.172085,3.449137,1.514412
362705,2.382159,3.152983,3.781734,4.116747,3.705794,2.691655,3.053596,2.723949,5.566973,1.578216,4.448726,3.945896
...,...,...,...,...,...,...,...,...,...,...,...,...
329881,2.382159,4.022782,1.474296,3.916025,4.234190,2.881626,4.969565,4.549766,2.091103,3.177830,3.376703,2.398588
56993,3.244274,3.442916,3.781734,2.260068,2.076573,2.359206,2.310085,2.637005,3.784928,2.746796,4.101043,1.956500
49024,2.382159,4.022782,3.637520,3.890934,3.782852,2.549177,3.968685,3.767273,2.144035,2.986259,3.898228,1.514412
12016,2.382159,3.152983,3.637520,2.435699,2.957233,2.549177,2.453068,2.376174,6.237445,1.319596,4.137260,4.830072


In [0]:
# normilize train and test data using z-score and shift it to avoid negative numbers
# use the same normalization pararameters for train and test datasets
# P.S. if u know how to avoid the warnings tell me please

In [0]:
# split training data into training (75%) and validation (25%) datasets

In [0]:
x = train_data[["year", "month", "hour", "PM2.5", "PM10", "SO2", "NO2", "CO", "O3", "PRES", "DEWP", "wd"]]
y = train_label
validation_data = pd.DataFrame()
train_data, validation_data, train_label, validation_label = train_test_split(x, y, test_size=0.25)

In [0]:
train_data

,year,month,hour,PM2.5,PM10,SO2,NO2,CO,O3,PRES,DEWP,wd
374201,3.244274,4.312715,3.781734,2.134616,1.966490,2.359206,2.567454,2.376174,2.567492,3.924954,2.768258,1.735456
267447,3.244274,3.732849,3.493305,2.209887,2.032540,2.359206,1.852540,2.289231,4.755349,2.804267,3.941688,3.503808
246314,1.520044,2.283184,1.618511,2.046800,1.922457,2.406699,1.909733,2.202287,3.326184,3.647177,2.051162,2.177544
323743,2.382159,1.703319,2.339585,2.046800,2.098589,2.881626,2.681840,2.376174,2.691000,4.614608,0.732865,2.840676
106332,1.520044,2.283184,3.060660,2.473335,2.626985,2.881626,2.481664,2.463118,3.484980,1.961357,2.587173,1.956500
...,...,...,...,...,...,...,...,...,...,...,...,...
22061,3.244274,3.732849,2.051155,2.096981,1.966490,2.359206,1.852540,2.202287,3.379116,2.488176,4.035852,3.061720
31300,4.106389,3.732849,1.906940,5.107812,4.575446,2.359206,3.196578,2.984780,2.549848,2.066721,4.195207,2.398588
56421,3.244274,3.442916,4.358594,2.272613,2.120606,2.359206,2.910613,2.810893,3.749640,2.957524,4.231424,1.735456
73185,1.520044,3.152983,2.628015,2.674057,3.177398,2.881626,2.424471,2.810893,3.573200,1.319596,4.369048,4.166940


In [0]:
validation_data

,year,month,hour,PM2.5,PM10,SO2,NO2,CO,O3,PRES,DEWP,wd
153973,2.382159,3.732849,3.204875,2.347883,2.175647,2.359206,2.567454,2.637005,2.479272,3.216144,3.992392,1.514412
178335,1.520044,3.152983,3.493305,2.285158,2.186655,2.881626,2.481664,2.202287,4.967077,0.620364,4.202450,4.166940
304504,3.244274,4.312715,3.637520,2.134616,2.032540,2.406699,1.995523,2.115343,2.849796,4.595451,1.363040,4.609028
194365,3.244274,2.573117,3.204875,2.222432,2.043548,2.359206,1.966926,2.115343,3.784928,2.938366,2.493009,2.177544
96236,4.106389,1.703319,4.214379,3.276223,3.661761,3.689001,3.139385,2.897836,2.549848,3.417293,2.174300,1.956500
...,...,...,...,...,...,...,...,...,...,...,...,...
184072,2.382159,1.703319,3.637520,3.276223,2.704043,4.876318,3.053596,2.810893,2.955660,3.589706,2.594417,2.398588
383323,4.106389,4.312715,4.070164,2.096981,1.933465,2.454192,1.966926,2.289231,3.043880,4.901964,1.536881,2.177544
72315,1.520044,2.573117,1.762726,2.623876,2.902192,3.594016,2.310085,2.637005,3.749640,1.424960,3.434651,2.398588
52386,3.244274,1.703319,3.925949,3.840754,3.463612,2.834133,3.110789,3.593386,2.055815,3.465185,2.855179,4.166940


In [0]:
clf = svm.SVR() # create the regression svm model

In [0]:
clf.fit(train_data, train_label) #  train the model

In [0]:
filename = '/content/drive/My Drive/Colab Notebooks/svm_model_2.0.sav'
#pickle.dump(clf, open(filename, 'wb'))

with open('/content/drive/My Drive/Colab Notebooks/svm_model_2.0.sav', 'wb') as f:
    pickle.dump(clf, f)


In [0]:
loaded_model = pickle.load(open(filename, 'rb'))  # open the saved model

FileNotFoundError: ignored

In [0]:
accuracy = loaded_model.score(validation_data, validation_label)  # check accuracy on validation dataset
print(accuracy)

0.9606084028547748


In [0]:
accuracy = loaded_model.score(test_data, test_label)  # check accuracy on test dataset
print(accuracy)

0.9613762898247007


In [0]:
temp_predicted = loaded_model.predict(test_data)  # calculate predictions for test dataset

In [0]:
df_test = pd.DataFrame({'Actual': test_label, 'Predicted': temp_predicted})  # create dataset with predictions

In [0]:
df_test

,Actual,Predicted
34750,1.800000,2.798618
362698,26.200000,28.173268
79780,9.900000,9.459159
52797,0.600000,-2.281623
277487,10.900000,10.847745
...,...,...
59656,0.900000,2.229934
342966,7.300000,6.119887
356457,13.700000,12.014116
385697,15.000000,13.345629


In [0]:
#  add categorization labels

In [0]:
group_names = ['verycold', 'cold', 'moderate', 'hot', 'veryhot']
bins = [-30, 0, 10, 20, 30, 50]
df_test["Actual_category"] = pd.cut(df_test["Actual"], bins = bins, include_lowest=True, labels = group_names)
df_test["Predicted_category"] = pd.cut(df_test["Predicted"], bins = bins, include_lowest=True, labels = group_names)

In [0]:
df_test  # show the result

,Actual,Predicted,Actual_category,Predicted_category
34750,1.800000,2.798618,cold,cold
362698,26.200000,28.173268,hot,hot
79780,9.900000,9.459159,cold,cold
52797,0.600000,-2.281623,cold,verycold
277487,10.900000,10.847745,moderate,moderate
...,...,...,...,...
59656,0.900000,2.229934,cold,cold
342966,7.300000,6.119887,cold,cold
356457,13.700000,12.014116,moderate,moderate
385697,15.000000,13.345629,moderate,moderate
